# Импортируем библиотеки

In [82]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re
from datetime import datetime, time
import os
import json
import re
from collections import Counter
import matplotlib
import matplotlib.pyplot as plt
import ast
import string
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy.stats import boxcox
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error, make_scorer
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import StackingRegressor

# Займемся моделями

In [83]:
# Импортируем предподготовленный ДФ
df = pd.read_csv('SF_DS_5_prepair.csv')
df = df.drop('Unnamed: 0', axis=1)

## Создадим dummy-переменные для категориальных признаков

In [84]:
# Запишем категориальные переменные
cat = ['body', 'brand', 'color', 'fuel', 'model', 'transmition', 'vendor', 'drive_type']
# Получим дамми переменные
df[pd.get_dummies(df[cat]).columns] = pd.get_dummies(df[cat])
# Удалим категориальные переменные
df.drop(cat, axis = 1, inplace = True)

## Приступим к оформлению тренировочного и тестового ДФ

In [85]:
df_test_kaggle = df[df['is_test'] == 1]
df_test_kaggle = df_test_kaggle.drop(['price', 'real_price'], axis = 1)

In [86]:
df = df[df['is_test'] == 0]

## Разделим выборку (df) на тренировочную и валидационную

In [87]:
num_col = ['V_engine', 'power', 'mileage', 'model_date', 'doors', 'date', 'owners', 'clearance', 'acceleration', 
          'word_in_description']

In [88]:
# Удалим колонку "real_price"
real_price = df['real_price']
real_price.index = df.index
df.drop('real_price', axis = 1, inplace = True)
# Разделим на целевую переменную и признаки
y = df['price']
X = df.drop(['price','sell_id'], axis = 1)
# Стандартизируем и нормализуем числовые признаки
st_scaler = StandardScaler()
X[num_col] = st_scaler.fit_transform(X[num_col])
norm_scaler = MinMaxScaler()
X[num_col] = norm_scaler.fit_transform(X[num_col])
# Разделим на валидационную и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## Метрикой будет служить MAPE
Средняя абсолютная процентная ошибка

### Обучим линейную регерссию
Возьмём ее за базовую

In [9]:
lin_reg = LinearRegression().fit(X_train, y_train)
y_pred_train = lin_reg.predict(X_train)
print('MAPE для тренировочной выборки на логарифмированной цене -', mean_absolute_percentage_error(y_train, y_pred_train))
print('MAPE для тренировочной выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_train.index], np.exp(y_pred_train)))
y_pred_test = lin_reg.predict(X_test)
print('MAPE для тестовой выборки на логарифмированной цене -', mean_absolute_percentage_error(y_test, y_pred_test))
print('MAPE для тестовой выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_test.index], np.exp(y_pred_test)))

MAPE для тренировочной выборки на логарифмированной цене - 0.016428160446833984
MAPE для тренировочной выборки на нормальной цене - 0.2174277379152257
MAPE для тестовой выборки на логарифмированной цене - 0.016614402320076896
MAPE для тестовой выборки на нормальной цене - 0.2200930242254463


### Обучим линейную регрессию только на числовых признаках

In [9]:
lin_reg_num = LinearRegression().fit(X_train[num_col], y_train)
y_pred_train = lin_reg_num.predict(X_train[num_col])
print('MAPE для тренировочной выборки на логарифмированной цене -', mean_absolute_percentage_error(y_train, y_pred_train))
print('MAPE для тренировочной выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_train.index], np.exp(y_pred_train)))
y_pred_test = lin_reg_num.predict(X_test[num_col])
print('MAPE для тестовой выборки на логарифмированной цене -', mean_absolute_percentage_error(y_test, y_pred_test))
print('MAPE для тестовой выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_test.index], np.exp(y_pred_test)))

MAPE для тренировочной выборки на логарифмированной цене - 0.024594452467426683
MAPE для тренировочной выборки на нормальной цене - 0.3367755131568577
MAPE для тестовой выборки на логарифмированной цене - 0.024779606716908546
MAPE для тестовой выборки на нормальной цене - 0.34051911582262606


### Добавим признаков

In [89]:
X['doors_owners'] = X['doors'] * X['owners']
X['V_engine_mileage'] = X['V_engine'] * X['mileage']
X['model_date_date'] = X['date'] - X['model_date'] 
X['clearance_acc'] = X['clearance']*X['acceleration']
X['mileage_power'] = X['mileage']*X['power']
X['V_engine_power'] = X['V_engine'] * X['power']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

### Заново обучим линейную регерссию

In [11]:
lin_reg = LinearRegression().fit(X_train, y_train)
y_pred_train = lin_reg.predict(X_train)
print('MAPE для тренировочной выборки на логарифмированной цене -', mean_absolute_percentage_error(y_train, y_pred_train))
print('MAPE для тренировочной выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_train.index], np.exp(y_pred_train)))
y_pred_test = lin_reg.predict(X_test)
print('MAPE для тестовой выборки на логарифмированной цене -', mean_absolute_percentage_error(y_test, y_pred_test))
print('MAPE для тестовой выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_test.index], np.exp(y_pred_test)))

MAPE для тренировочной выборки на логарифмированной цене - 0.016311601873839468
MAPE для тренировочной выборки на нормальной цене - 0.21568618507572754
MAPE для тестовой выборки на логарифмированной цене - 0.01650764106090205
MAPE для тестовой выборки на нормальной цене - 0.21839409959910525


### Обучим лес

In [12]:
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X_train, y_train)
y_pred_train = tree_reg.predict(X_train)
print('MAPE для тренировочной выборки на логарифмированной цене -', mean_absolute_percentage_error(y_train, y_pred_train))
print('MAPE для тренировочной выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_train.index], np.exp(y_pred_train)))
y_pred_test = tree_reg.predict(X_test)
print('MAPE для тестовой выборки на логарифмированной цене -', mean_absolute_percentage_error(y_test, y_pred_test))
print('MAPE для тестовой выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_test.index], np.exp(y_pred_test)))
print(tree_reg.get_params())
print('depth -', tree_reg.get_depth())
print('n_leaves -', tree_reg.get_n_leaves())

MAPE для тренировочной выборки на логарифмированной цене - 6.537852652666519e-05
MAPE для тренировочной выборки на нормальной цене - 0.0009063418089238547
MAPE для тестовой выборки на логарифмированной цене - 0.013228811501296163
MAPE для тестовой выборки на нормальной цене - 0.17574336042038124
{'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 42, 'splitter': 'best'}
depth - 50
n_leaves - 145483


In [13]:
tree_reg = DecisionTreeRegressor(random_state=42, max_depth = 15)
tree_reg.fit(X_train, y_train)
y_pred_train = tree_reg.predict(X_train)
print('MAPE для тренировочной выборки на логарифмированной цене -', mean_absolute_percentage_error(y_train, y_pred_train))
print('MAPE для тренировочной выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_train.index], np.exp(y_pred_train)))
y_pred_test = tree_reg.predict(X_test)
print('MAPE для тестовой выборки на логарифмированной цене -', mean_absolute_percentage_error(y_test, y_pred_test))
print('MAPE для тестовой выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_test.index], np.exp(y_pred_test)))

MAPE для тренировочной выборки на логарифмированной цене - 0.009850894206516041
MAPE для тренировочной выборки на нормальной цене - 0.12603172297405207
MAPE для тестовой выборки на логарифмированной цене - 0.012576974440306527
MAPE для тестовой выборки на нормальной цене - 0.1645598923465765


### Случайный лес

In [10]:
# Создадим сетку для поиска лучших параметров
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 50, num = 10)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [11]:
#rf = RandomForestRegressor(random_state=42)
#rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=10, 
#                               cv=3, verbose=2, random_state=42, n_jobs=-1)
#rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42),
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 14, 18, 23, 27, 32,
                                                      36, 41, 45, 50, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [10, 14, 19, 24, 28, 33,
                                                         38, 43, 47, 52, 57, 62,
                                                         66, 71, 76, 81, 85, 90,
                                                         95, 100]},
                   random_state=42, verbose=2)

In [12]:
#rf_random.best_params_

{'n_estimators': 38,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 23,
 'bootstrap': False}

In [14]:
rf_best = RandomForestRegressor(random_state=42, n_estimators = 38, min_samples_split = 2, max_features = 'sqrt',
                          max_depth = 23, bootstrap = False, n_jobs = -1)
rf_best.fit(X_train, y_train)
y_pred_train = rf_best.predict(X_train)
print('MAPE для тренировочной выборки на логарифмированной цене -', mean_absolute_percentage_error(y_train, y_pred_train))
print('MAPE для тренировочной выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_train.index], np.exp(y_pred_train)))
y_pred_test = rf_best.predict(X_test)
print('MAPE для тестовой выборки на логарифмированной цене -', mean_absolute_percentage_error(y_test, y_pred_test))
print('MAPE для тестовой выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_test.index], np.exp(y_pred_test)))

MAPE для тренировочной выборки на логарифмированной цене - 0.004644223332718792
MAPE для тренировочной выборки на нормальной цене - 0.058043638468342575
MAPE для тестовой выборки на логарифмированной цене - 0.01019724767748368
MAPE для тестовой выборки на нормальной цене - 0.13088936854630495


### ADA Boost

In [18]:
ab_reg = AdaBoostRegressor(base_estimator = RandomForestRegressor(random_state=42, n_estimators = 38, min_samples_split = 2, max_features = 'sqrt',
                          max_depth = 23, bootstrap = False, n_jobs = -1), random_state=42, n_estimators=10, loss = 'linear')

In [19]:
ab_reg.fit(X_train, y_train)
y_pred_train = ab_reg.predict(X_train)
print('MAPE для тренировочной выборки на логарифмированной цене -', mean_absolute_percentage_error(y_train, y_pred_train))
print('MAPE для тренировочной выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_train.index], np.exp(y_pred_train)))
y_pred_test = ab_reg.predict(X_test)
print('MAPE для тестовой выборки на логарифмированной цене -', mean_absolute_percentage_error(y_test, y_pred_test))
print('MAPE для тестовой выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_test.index], np.exp(y_pred_test)))

MAPE для тренировочной выборки на логарифмированной цене - 0.005022662360787289
MAPE для тренировочной выборки на нормальной цене - 0.062481492311116764
MAPE для тестовой выборки на логарифмированной цене - 0.010208585748426
MAPE для тестовой выборки на нормальной цене - 0.13053956012493448


### Gradient boosting

In [21]:
gb_reg = GradientBoostingRegressor()

In [22]:
gb_reg.fit(X_train, y_train)
y_pred_train = gb_reg.predict(X_train)
print('MAPE для тренировочной выборки на логарифмированной цене -', mean_absolute_percentage_error(y_train, y_pred_train))
print('MAPE для тренировочной выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_train.index], np.exp(y_pred_train)))
y_pred_test = gb_reg.predict(X_test)
print('MAPE для тестовой выборки на логарифмированной цене -', mean_absolute_percentage_error(y_test, y_pred_test))
print('MAPE для тестовой выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_test.index], np.exp(y_pred_test)))

MAPE для тренировочной выборки на логарифмированной цене - 0.015561522476176131
MAPE для тренировочной выборки на нормальной цене - 0.20424352465474505
MAPE для тестовой выборки на логарифмированной цене - 0.015782679659450946
MAPE для тестовой выборки на нормальной цене - 0.20726961483884554


In [37]:
# Создадим сетку для поиска лучших параметров
learning_rate = [int(x)/10 for x in np.linspace(start = 1, stop = 10, num = 5)]
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
max_depth.append(None)
max_features = ['auto', 'sqrt']
alpha = [int(x)/10 for x in np.linspace(start = 1, stop = 9, num = 5)]
random_grid = {'learning_rate': learning_rate,
               'n_estimators': n_estimators,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,               
               'max_depth': max_depth,
               'max_features': max_features,
               'alpha': alpha}

In [38]:
rf = GradientBoostingRegressor(random_state=42)
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=10, 
                               cv=3, verbose=2, random_state=42, n_jobs=-1)
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=GradientBoostingRegressor(random_state=42),
                   n_jobs=-1,
                   param_distributions={'alpha': [0.1, 0.3, 0.5, 0.7, 0.9],
                                        'learning_rate': [0.1, 0.3, 0.5, 0.7,
                                                          1.0],
                                        'max_depth': [10, 20, 30, 40, 50, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [10, 20, 30, 40, 50, 60,
                                                         70, 80, 90, 100]},
                   random_state=42, verbose=2)

In [39]:
rf_random.best_params_

{'n_estimators': 90,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 30,
 'learning_rate': 0.1,
 'alpha': 0.9}

In [44]:
gb_reg = GradientBoostingRegressor(n_estimators = 90, min_samples_split = 2, min_samples_leaf = 4, max_features = 'sqrt',
                                  max_depth = 30, learning_rate = 0.1, alpha = 0.9)

In [45]:
gb_reg.fit(X_train, y_train)
y_pred_train = gb_reg.predict(X_train)
print('MAPE для тренировочной выборки на логарифмированной цене -', mean_absolute_percentage_error(y_train, y_pred_train))
print('MAPE для тренировочной выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_train.index], np.exp(y_pred_train)))
y_pred_test = gb_reg.predict(X_test)
print('MAPE для тестовой выборки на логарифмированной цене -', mean_absolute_percentage_error(y_test, y_pred_test))
print('MAPE для тестовой выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_test.index], np.exp(y_pred_test)))

MAPE для тренировочной выборки на логарифмированной цене - 0.003097334815117891
MAPE для тренировочной выборки на нормальной цене - 0.03859653200926517
MAPE для тестовой выборки на логарифмированной цене - 0.010216172733756893
MAPE для тестовой выборки на нормальной цене - 0.1312034513609127


### Stacking

In [55]:
estimators = [
     ('lr', RidgeCV()),
     ('svr', LinearSVR(random_state=42)),
     ('gb', GradientBoostingRegressor(n_estimators = 90, min_samples_split = 2, min_samples_leaf = 4, max_features = 'sqrt',
                                  max_depth = 30, learning_rate = 0.1, alpha = 0.9)),
     ('rf', RandomForestRegressor(random_state=42, n_estimators = 38, min_samples_split = 2, max_features = 'sqrt',
                          max_depth = 23, bootstrap = False, n_jobs = -1))
]
st_reg = StackingRegressor(
     estimators=estimators,
     final_estimator=RandomForestRegressor(n_estimators=10,
                                           random_state=42), n_jobs = -1)

In [54]:
st_reg.fit(X_train, y_train)
y_pred_train = st_reg.predict(X_train)
print('MAPE для тренировочной выборки на логарифмированной цене -', mean_absolute_percentage_error(y_train, y_pred_train))
print('MAPE для тренировочной выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_train.index], np.exp(y_pred_train)))
y_pred_test = st_reg.predict(X_test)
print('MAPE для тестовой выборки на логарифмированной цене -', mean_absolute_percentage_error(y_test, y_pred_test))
print('MAPE для тестовой выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_test.index], np.exp(y_pred_test)))

C:\Users\kosty\programms\anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\kosty\programms\anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\kosty\programms\anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\kosty\programms\anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\kosty\programms\anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase th

MAPE для тренировочной выборки на логарифмированной цене - 0.006314018875961251
MAPE для тренировочной выборки на нормальной цене - 0.07901886155786951
MAPE для тестовой выборки на логарифмированной цене - 0.0111292721423741
MAPE для тестовой выборки на нормальной цене - 0.14295029017388763


In [90]:
  estimators = [
     ('gb', GradientBoostingRegressor(n_estimators = 90, min_samples_split = 2, min_samples_leaf = 4, max_features = 'sqrt',
                                  max_depth = 30, learning_rate = 0.1, alpha = 0.9)),
     ('rf', RandomForestRegressor(random_state=42, n_estimators = 38, min_samples_split = 2, max_features = 'sqrt',
                          max_depth = 23, bootstrap = False, n_jobs = -1))
]
st_reg = StackingRegressor(
     estimators=estimators,
    n_jobs = -1
)

In [91]:
st_reg.fit(X_train, y_train)
y_pred_train = st_reg.predict(X_train)
print('MAPE для тренировочной выборки на логарифмированной цене -', mean_absolute_percentage_error(y_train, y_pred_train))
print('MAPE для тренировочной выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_train.index], np.exp(y_pred_train)))
y_pred_test = st_reg.predict(X_test)
print('MAPE для тестовой выборки на логарифмированной цене -', mean_absolute_percentage_error(y_test, y_pred_test))
print('MAPE для тестовой выборки на нормальной цене -', mean_absolute_percentage_error(real_price.loc[y_test.index], np.exp(y_pred_test)))

MAPE для тренировочной выборки на логарифмированной цене - 0.003208975030116997
MAPE для тренировочной выборки на нормальной цене - 0.039860283129458246
MAPE для тестовой выборки на логарифмированной цене - 0.009941229642810693
MAPE для тестовой выборки на нормальной цене - 0.12764768978209967


# Лучшая модель - последний стэкинг

### Сделаем предсказание

In [92]:
sell_id = df_test_kaggle['sell_id']
df_test_kaggle.drop('sell_id', axis = 1, inplace = True)

In [93]:
# Стандартизируем и нормализуем датасет
#st_scaler = StandardScaler()
df_test_kaggle[num_col] = st_scaler.transform(df_test_kaggle[num_col])
#norm_scaler = MinMaxScaler()
df_test_kaggle[num_col] = norm_scaler.transform(df_test_kaggle[num_col])

In [94]:
# Добавим переменные
df_test_kaggle['doors_owners']     = df_test_kaggle['doors'] *    df_test_kaggle['owners']
df_test_kaggle['V_engine_mileage'] = df_test_kaggle['V_engine'] * df_test_kaggle['mileage']
df_test_kaggle['model_date_date']  = df_test_kaggle['date'] -     df_test_kaggle['model_date'] 
df_test_kaggle['clearance_acc']    = df_test_kaggle['clearance']* df_test_kaggle['acceleration']
df_test_kaggle['mileage_power']    = df_test_kaggle['mileage']*   df_test_kaggle['power']
df_test_kaggle['V_engine_power']   = df_test_kaggle['V_engine'] * df_test_kaggle['power']

In [97]:
# Сделаем предсказание
y_ans = st_reg.predict(df_test_kaggle)
y_ans = np.exp(y_ans)
y_ans = np.around(y_ans)
y_ans

array([ 682051.,  976824.,  925885., ...,  224676., 1231048., 1121577.])

In [98]:
ans = pd.DataFrame()
ans['sell_id'] = sell_id
ans['price'] = y_ans
ans

,sell_id,price
213019,1100575026,682051.0
213020,1100549428,976824.0
213021,1100658222,925885.0
213022,1100937408,844231.0
213023,1101037972,882611.0
...,...,...
247700,1101369263,1010436.0
247701,1101369581,2377358.0
247702,1101364889,224676.0
247703,1101362518,1231048.0


In [106]:
sample_submission = pd.read_csv('c:/Users/kosty/SF DS Project 5/sample_submission.csv', sep = ',', encoding = 'utf-8')

In [107]:
sample_submission

,sell_id,price
0,1100575026,0
1,1100549428,0
2,1100658222,0
3,1100937408,0
4,1101037972,0
...,...,...
34681,1101369263,0
34682,1101369581,0
34683,1101364889,0
34684,1101362518,0


In [112]:
mdf = ans.merge(sample_submission, on = 'sell_id')
mdf

Index(['sell_id', 'price_x', 'price_y'], dtype='object')

In [113]:
mdf.drop('price_y', axis = 1, inplace = True)
mdf.columns = ['sell_id', 'price']
mdf

,sell_id,price
0,1100575026,682051.0
1,1100549428,976824.0
2,1100658222,925885.0
3,1100937408,844231.0
4,1101037972,882611.0
...,...,...
34681,1101369263,1010436.0
34682,1101369581,2377358.0
34683,1101364889,224676.0
34684,1101362518,1231048.0


In [115]:
mdf.to_csv('sample_submission.csv', index = False)